Определение языка (language detection)
--------------------

In [1]:
import warnings
warnings.filterwarnings('ignore')

#### Скачаем тексты из википедии

In [2]:
import wikipedia

+ Выберем 10 языков.

In [3]:
languages = ['bg', 'be', 'kk', 'uk', 'fr', 'ru', 'en', 'mk', 'la', 'de']

In [4]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [5]:
wiki_texts = {}
for lang in languages:
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

Skipping page Тексас (пояснение)
Skipping page Евксиноград (пояснение)
Skipping page Арат (пояснение)
Skipping page Лапландия (пояснение)
Skipping page Келвин (пояснение)
Skipping page Алесандро Фарнезе
Skipping page Приют
Skipping page Британик (пояснение)
bg 92
Skipping page Міхаіл Загоскін (значэнні)
Skipping page Сташэўскі
Skipping page Мужанка (значэнні)
Skipping page Грамыкі
Skipping page Знание
be 95
kk 100
Skipping page Новоозерне
Skipping page Єнішовіце
Skipping page Абінгтон
Skipping page Петрик
uk 96
Skipping page Saint-Martin-Lars-en-Sainte-Hermine
Skipping page Lamu
Skipping page L'Invention de Morel
Skipping page Chabbie
fr 96
Skipping page Миркович
Skipping page Резерват
Skipping page Дурсун
Skipping page Уикс, Джон
Skipping page История версий Java
Skipping page Суперлига Б 2001/2002
Skipping page Демидовский район (Смоленская область)
Skipping page Лебедянка
Skipping page Вьюшков
Skipping page День работников государственной статистики (Белоруссия)
Skipping page Ананче

In [6]:
import re
import codecs
import collections
import sys


+ Дополнительно очистим текст от знаков перпинания, лишних пробелов/табуляцй/переносов, от цифр и тд.

In [7]:
def tokenize(text):
    text = text.lower()
    text = re.sub('(http://|https://|www.)[a-zA-z0-9]*? ', '', text)
    text = re.sub('[^\w\s]', '',text)
    text = re.sub('[\s]{2,}', ' ',text)
    text = re.sub('[0-9]+', '',text)
    return text.split(' ')


### Первый метод: частотные слова

Формируем частотный словарь (кортеж словарей, получается) для слов в языке.

Ключ -- это язык, а значение -- это словарь, где ключ -- это слово, а значение -- это частотность слова во всем словаре (корпусе) для одного языка.

In [114]:
freq_dict = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))

Сначала просто частотный словарь:

In [115]:
for lang in languages:
    for article in wiki_texts[lang]:
        for word in tokenize(article.replace('\n', '').lower()):
            freq_dict[lang][word] += 1

Потом меняем частоту на абсолютное значение.

In [116]:
for lang in languages:
    for word in freq_dict[lang]:
        freq_dict[lang][word] = freq_dict[lang][word]/sum(freq_dict[lang].values())

Посмотрим, что у нас там:)

In [22]:
for word in sorted(freq_dict['kk'], key=lambda w: freq_dict['kk'][w], reverse=True)[:10]:
    print('{}\t{}'.format(freq_dict['kk'][word], word))

0.24532825706742278	жалғастырғышты
0.12463899494783258	жүріс
0.11222148334254944	тежегішті
0.10194716438342984	жөне
0.09448440692641817	тісті
0.09332117678404195	түзілімді
0.08598678049100969	үйкеліс
0.07968118951576447	үйлестіргіштері
0.07263155251366435	берілістер
0.059803047545895646	қорабы


In [23]:
def predict_language(text, freq_dict):
    words_var = {}
    for lang in freq_dict.keys():
        count = 0
        for word in tokenize(text):
            if word in freq_dict[lang]:
                count += freq_dict[lang][word]
        words_var[lang] = count
    
    result = max(words_var, key=lambda x: words_var[x]) 
    
    return result

In [24]:
predict_language('ich liebe dich', freq_dict)

'de'

Однако, всё намного сложнее с языками, где присутствует кириллица:

In [26]:
predict_language('Привет. Как дела? Какой это язык, скажешь?', freq_dict)

'ru'

In [28]:
predict_language('Что это за язык, скажешь?', freq_dict)

'mk'

In [39]:
predict_language('Привет. определи язык, пожалуйста!', freq_dict)

'uk'

Не очень поняла, что подразумевается под "удалить повторения из словарей":(

Самым логичным трактованием этого задания мне показалось: удалить одинаковые единицы из словарей, то есть сделать разность словарей. В число удалений, скорей всего, войдут служебные слова у языков одной языковой семьи, а также возможные деривации, которые не деформировались в результате заимствования. Однако, мы можем и не удалять эти повторения из словарей, потому что они никак не должны повлиять на результаты определения языка.

In [45]:
from sklearn.metrics import classification_report, confusion_matrix


In [64]:
true_labels = []
predicted_labels = []

for lang in languages:
    for text in wiki_texts[lang]:
        true_labels.append(lang)
        predicted_labels.append(predict_language(text, freq_dict))
        

In [65]:
print(classification_report(true_labels, predicted_labels))

             precision    recall  f1-score   support

         be       1.00      0.91      0.95        95
         bg       0.69      0.98      0.81        92
         de       0.99      0.98      0.98        93
         en       0.91      0.99      0.95        96
         fr       0.95      0.99      0.97        96
         kk       0.99      1.00      1.00       100
         la       1.00      0.85      0.92        97
         mk       0.96      0.95      0.95        97
         ru       0.83      0.71      0.76        89
         uk       0.99      0.85      0.92        96

avg / total       0.93      0.92      0.92       951



In [68]:
print(confusion_matrix(true_labels, predicted_labels))

[[ 86   3   0   0   0   0   0   2   3   1]
 [  0  90   0   1   0   0   0   1   0   0]
 [  0   0  91   0   0   0   0   0   2   0]
 [  0   0   0  95   0   1   0   0   0   0]
 [  0   0   0   0  95   0   0   0   1   0]
 [  0   0   0   0   0 100   0   0   0   0]
 [  0   0   1   6   5   0  82   0   3   0]
 [  0   4   0   0   0   0   0  92   1   0]
 [  0  26   0   0   0   0   0   0  63   0]
 [  0   8   0   2   0   0   0   1   3  82]]


### Второй метод: частотные символьные n-граммы

Создадим функцию, которая преобразовывает строку в массив n-грамм заданной длины.

In [73]:
from itertools import islice, tee

def make_ngrams(text):
    text = text.lower()
    text = re.sub('(http://|https://|www.)[a-zA-z0-9]*? ', '', text)
    text = re.sub('[^\w\s]', '',text)
    text = re.sub('[\s]{2,}', ' ',text)
    text = re.sub('[0-9]+', '',text)
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return ngrams

In [74]:
make_ngrams('ich heiße Alina')

['ich',
 'ch ',
 'h h',
 ' he',
 'hei',
 'eiß',
 'iße',
 'ße ',
 'e a',
 ' al',
 'ali',
 'lin',
 'ina']

Теперь создадим частотные словари n-грамм аналогично первому методу.

In [82]:
freq_ndrm = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))

In [83]:
for lang in languages:
    for article in wiki_texts[lang]:
        for word in make_ngrams(article.replace('\n', '').lower()):
            freq_ndrm[lang][word] += 1

In [84]:
for lang in languages:
    for word in freq_ndrm[lang]:
        freq_ndrm[lang][word] = freq_ndrm[lang][word]/sum(freq_ndrm[lang].values())


Посмотрим, что у нас там.

In [85]:
for word in sorted(freq_ndrm['kk'], key=lambda w: freq_ndrm['kk'][w], reverse=True)[:10]:
    print('{}\t{}'.format(freq_ndrm['kk'][word], word))

0.12213749127777604	жиі
0.10104254400129953	жег
0.09255603943632926	теж
0.08533199900519951	үйл
0.07911506206267573	жұд
0.07371312515600513	ону
0.06897919002189508	жон
0.05774591435037658	үра
0.05475705951060114	зм 
0.05205505456591226	пқо


In [123]:
def ngram_predict_language(text, freq_ndrm):
    ngram_var = {}
    
    for lang in freq_ndrm.keys():
        count = 0
        for word in make_ngrams(text):
            if word in freq_ndrm[lang]:
                count += freq_ndrm[lang][word]
        ngram_var[lang] = count
    
    return max(ngram_var, key=lambda x: ngram_var[x])


In [124]:
ngram_predict_language(text, freq_ndrm)

'de'

In [125]:
ngram_predict_language('Привет. Как дела? Какой это язык, скажешь?', freq_ndrm)

'mk'

In [126]:
ngram_predict_language('Что это за язык, скажешь?', freq_ndrm)

'mk'

In [127]:
ngram_predict_language('Привет. определи язык, пожалуйста!', freq_ndrm)

'kk'

In [128]:
true_labels = []
predicted_labels = []

for lang in languages:
    for text in wiki_texts[lang]:
        true_labels.append(lang)
        predicted_labels.append(ngram_predict_language(text, freq_ndrm))
        

In [129]:
print(classification_report(true_labels, predicted_labels))

             precision    recall  f1-score   support

         be       1.00      0.79      0.88        95
         bg       0.65      0.33      0.43        92
         de       0.95      0.99      0.97        93
         en       0.97      0.97      0.97        96
         fr       0.98      1.00      0.99        96
         kk       0.95      0.99      0.97       100
         la       0.97      0.97      0.97        97
         mk       0.41      0.99      0.58        97
         ru       0.96      0.28      0.43        89
         uk       0.82      0.66      0.73        96

avg / total       0.87      0.80      0.80       951



In [130]:
print(confusion_matrix(true_labels, predicted_labels))

[[75  1  1  0  0  1  0  5  0 12]
 [ 0 30  0  1  0  0  0 61  0  0]
 [ 0  0 92  0  0  0  1  0  0  0]
 [ 0  0  0 93  2  0  0  1  0  0]
 [ 0  0  0  0 96  0  0  0  0  0]
 [ 0  1  0  0  0 99  0  0  0  0]
 [ 0  0  3  0  0  0 94  0  0  0]
 [ 0  0  0  0  0  0  0 96  1  0]
 [ 0  7  0  0  0  2  2 51 25  2]
 [ 0  7  1  2  0  2  0 21  0 63]]
